In [1]:
def jk_read(dataset='',var=None,njk=100,auto_cross='_cross',bins='_bins',
            home='',corr='_final',skiprows=0,**kwargs):
    data_jk={}
    filename=dataset
    try:
        data_jk['data']=np.genfromtxt(home+filename+bins+auto_cross+'_jk_final.dat',
                                      names=True,skip_header=skiprows)
    except:
        print ('file not found',home+filename+bins+auto_cross+'_jk_final.dat')
    try:
        data_jk['data0']=np.genfromtxt(home+filename+bins+auto_cross+corr+'.dat',
                                       names=True,skip_header=skiprows)
    except:
        print ('file not found',home+filename+bins+auto_cross+corr+'.dat')
    for i in np.arange(njk):
        data_jk[i]=np.genfromtxt(home+filename+bins+auto_cross+'_jk'+str(i)+corr+'.dat',names=True,
                                 skip_header=skiprows)
    cc=cov_corr()
    if var and njk>0:
        data_jk['cov']={}
        data_jk['corr']={}
        for v in var:
            cov,corr=cc.dict_cov(dic=data_jk,var=v,stack=1,ignore_keys=['data','data0','cov','corr'])
            cov*=njk-1.
            data_jk['cov'][v]=cov
            data_jk['corr'][v]=corr
    return data_jk


In [2]:
file_home='./temp/hsc_test/corr_data_out/'
file_name='hsc_test'

In [3]:
njk=0

In [4]:
w_jk=jk_read(dataset=file_name,var=['wgg','wgp'],njk=njk,auto_cross='_cross',bins='_bins',
            home=file_home,corr='_final',skiprows=0)

file not found ./temp/hsc_test/corr_data_out/hsc_test_bins_cross_jk_final.dat


In [5]:
w_jk.keys()

dict_keys(['data0'])

In [6]:
w_jk['data0']

array([(0.129245,    3328., 0., 0.,  0.00934529,  0., -0.00680549, -0., 0., 0., 0., 0., 0., 0., -6.88021, -0., 0.,    3328., 0.1     ,  0.158489, 0.),
       (0.204839,    7185., 0., 0., -0.00258386, -0.,  0.00872609,  0., 0., 0., 0., 0., 0., 0., -6.86842, -0., 0.,    7185., 0.158489,  0.251189, 0.),
       (0.324648,   15355., 0., 0., -0.00316065, -0., -0.00174473, -0., 0., 0., 0., 0., 0., 0., -6.90942, -0., 0.,   15355., 0.251189,  0.398107, 0.),
       (0.514532,   31891., 0., 0., -0.00308426, -0.,  0.00314672,  0., 0., 0., 0., 0., 0., 0., -6.9076 , -0., 0.,   31891., 0.398107,  0.630957, 0.),
       (0.815479,   67748., 0., 0., -0.00120796, -0., -0.00215722, -0., 0., 0., 0., 0., 0., 0., -6.9079 , -0., 0.,   67748., 0.630957,  1.      , 0.),
       (1.29245 ,  132112., 0., 0.,  0.00112028,  0.,  0.00357882,  0., 0., 0., 0., 0., 0., 0., -6.90481, -0., 0.,  132112., 1.      ,  1.58489 , 0.),
       (2.04839 ,  255198., 0., 0.,  0.0012611 ,  0., -0.0005873 , -0., 0., 0., 0., 0., 0., 0.

In [7]:
w_jk['data0']['npairs']

array([   3328.,    7185.,   15355.,   31891.,   67748.,  132112.,
        255198.,  513393., 1002919., 1746213.])

In [8]:
w_jk['data0']['wgp']

array([ 0.00934529, -0.00258386, -0.00316065, -0.00308426, -0.00120796,
        0.00112028,  0.0012611 ,  0.00048746, -0.000622  , -0.00086452])

'data': contains the jackknife mean and the jackknife errorbars

'data0': Contains the full sample measurements. Do not use the errorbars from this set.

0....njk-1: Contains measurements from the respective jackknife regions.

cov, corr: Covariance and correlation matrices for the variables passed.